## **MoMA Collection Data Processing**
### Table of Contents
1. [General EDA & Initial Cleaning](#general)   
2. [Formatting review](#formatting-review)
3. [Standardizing Values](#standardizing-values)   
a. [Removing punctuation & special characters](#punctuation)   
b. [Cleaning Dates](#dates)
4. [Extracting from Biographies](#bio-extraction)   
a. [Extracting birthplace](#birthplace)   
b. [Imputing nationalities](#impute-nationalities)   
c. [Imputing Deceased Year & Creating `living` flag](#creating-living-flag)   
5. [Data Type Corrections](#datatypes)
6. [Title EDA](#title-eda)

For the purposes of this project, the data will be limited to records with a singular artist listed as the creator of the artwork.   
All work done to process data for multi-artist pieces has been moved to [The Graveyard](#graveyard)   
a. [Evaluating `nulls`](#nulls)  
b. [Deduping within values](#deduping-within)   


### Data Type Corrections <a class='anchor' id='datatypes'></a>

### General Cleaning & EDA <a class='anchor' id='general'></a>
#### Dropped:
- Records missing `Artist` (1,216 records representing 0.8% of the entire catalogue)
- Records with multiple artists listed in `Artist` (9,577 records representing 6.8% of the entire catalogue)
  - Includes design groups/firms

#### Remaining dataset used contains **131,271 artworks** from **10,875 artists**

In [223]:
from clean_moma import *

import pandas as pd

original_catalogue = pd.read_csv('./data/MoMA data/Artworks.csv')
# any values without an artist listed will not be useful, setting those aside for potential investigation in the future
missing_artists = original_catalogue[original_catalogue.Artist.isnull()==True].copy()
artists_listed = original_catalogue.dropna(axis=0, subset=['Artist']).copy()
# print(len(missing_artists)/(len(original_catalogue)+len(missing_artists))), print(len(missing_artists))
# removing works with multiple artists/groups credited
catalogue = artists_listed[(artists_listed.Artist.str.contains(',|Associate|Architect', regex=True) == False)].copy()
# print((len(original_catalogue)-len(catalogue))/len(original_catalogue)), print(len(original_catalogue)-len(catalogue))

movements = pd.read_csv('./data/WikiArt Data/movements_by_artist.csv')

catalogue.drop(columns=['URL','ThumbnailURL','Circumference (cm)','Depth (cm)','Diameter (cm)','Weight (kg)','Seat Height (cm)'], inplace=True) # dropping unneeded columns
# [print(f"| {i} | {catalogue[i].dtype} |  |") for i in catalogue.columns]

### Content & Formatting Review <a class='anchor' id='formatting-review'></a>
### Artist Information
| Column | DataType | Notes |
| --- | --- | --- |
| ConstituentID | object | `int` |
| Artist | object | Investigate falls non-nulls (e.g. "Unknown" / "Unidentified"...) |
| ArtistBio | object | Extract `birthplace` & `birthyear`; create `living` flag |
| Gender | object | Clean punctuation & dedupe |
| Nationality | object | Clean punctuation & dedupe |
### Artwork Information
| Column | DataType | Notes |
| --- | --- | --- |
| AccessionNumber | object |  |
| ObjectID | int64 |  |
| Title | object | Create `untitled` flag |
| Medium | object | Investigate overlap w/`Classification` |
| Classification | object | Investigate overlap w/`Medium` |
| Dimensions | object |  |
| Height (cm) | object |  |
| Length (cm) | object |  |
| Width (cm) | object |  |
| Duration (sec.) | object | `int` |
### Dates
| Column | DataType | Notes |
| --- | --- | --- |
| BeginDate | object | `int`; Clean punctuation & standardize |
| EndDate | object | `int`; Clean punctuation & standardize |
| Date | object | `int`; Clean punctuation & standardize` |
### Institutional Data
| Column | DataType | Notes |
| --- | --- | --- |
| CreditLine | object | Investigate potentially meaningful nulls |
| Department | object |  |
| DateAcquired | object | `int`; Clean punctuation & standardize |
| Cataloged | object | `bool` |

In [224]:
# low hanging fruit
catalogue['Cataloged'].replace({'Y':True,'N':False}, inplace=True) # converting Catalogued to boolean
catalogue['Untitled'] = catalogue['Title'].str.contains('Untitled') # creating Untitled flag

### Standardizing Within Rows <a class="anchor" id="standardizing-within"></a>
#### Cleaning Punctuation <a class="anchor" id="punctuation"></a>

In [225]:
# datasets = {
#     'moma': moma,
#     'wikiart': wa,
#     'agp': agp
# }

punctuation = [
    {
        'columns': ['BeginDate','EndDate','Date'],
        'pattern': '[^0-9\-]'
    },
    {
        'columns': 'Nationality',
        'pattern': '[^A-Za-z0-9\,\-\s]'
    },
    {
        'columns': 'ArtistBio',
        'pattern': '[\(\)]'
    },
    {
        'columns': 'Gender',
        'pattern': '[^A-Za-z\-\s]'
    }
    
]

for item in punctuation:
    strip_punct(catalogue, item['columns'], item['pattern'])
    
catalogue['Gender'] = catalogue['Gender'].apply(lambda x: x.capitalize())
catalogue.fillna('-', inplace=True) # filling nulls for easier review

### Cleaning Dates <a class="anchor" id="dates"></a>
75% of `Date` records were complete & correctly formatted (98,541 of 131,271 records).   
Achieved 23% improvement (30,269 records) with `format_date` function.   
     
3,148 or 2% of records are missing `Date` value **_see_**: [date imputation](#date-imputation)


In [236]:
catalogue['date_cleaned'] = catalogue['Date'].apply(lambda x: format_date(x)[0]) # processing & overwriting date column
catalogue['date2'] = catalogue['Date'].apply(lambda x: format_date(x)[1]) # storing additional data in secondary field for later review

In [190]:
## original code & notes for improvement measurements
# # clean_dates = catalogue[(catalogue.Date.str.len()==4)&(catalogue.Date.str.contains('\D')==False)].index
# catalogue.loc[clean_dates,'cleanedDate'] = catalogue.loc[clean_dates,'Date']
# integrity review printout
# print('{:,} total rows \n{:,} rows have dates correctly formatted'.format(len(catalogue), len(catalogue[catalogue.Date.str.len()==4])))
# print(f'completeness of Date {len(clean_dates)/(len(catalogue)):.0%}\n')

# catalogue[catalogue.Date.str.len()!=4]['Date'].value_counts()[0:2]; #missing values
# catalogue[catalogue.Date.str.len()!=4]['Date'].value_counts()[2:30]; # identifying most common, alternate Date formatting

# cleaning date ranges yyyyyyyy and yyyyyy
# create index reference
# eight_digits = catalogue[((catalogue.Date.str.len()==6)|(catalogue.Date.str.len()==8))&(catalogue.Date.str.contains('\D',regex=True)==False)].index
# apply processing function to rows of given index
# store formatted year in cleanedDate, remaining data in Date2 for later review
# catalogue.loc[eight_digits,'Date2'] = catalogue.loc[eight_digits,'Date'].apply(lambda x: str(x)[4:])
# catalogue.loc[eight_digits,'cleanedDate'] = catalogue.loc[eight_digits,'Date'].apply(lambda x: str(x)[0:4])
# print(f'YYYYYYYY & YYYYYY formatting: completeness of Date improved {len(eight_digits)/(len(catalogue)):.0%}')

# pulling all dates with ranges yyyy-yyyy | yyyy-yy
# create index reference
# dash_ranges = catalogue[((catalogue.Date.str.len()==7)|(catalogue.Date.str.len()==9))&(catalogue.Date.str.contains('\d{4}\-\d{1,4}',regex=True))].index
# apply processing function to rows of given index
# store formatted year in cleanedDate, remaining data in Date2 for later review
# catalogue.loc[dash_ranges,'Date2'] = catalogue.loc[dash_ranges,'Date'].apply(lambda x: x.split('-')[1])
# catalogue.loc[dash_ranges,'cleanedDate'] = catalogue.loc[dash_ranges,'Date'].apply(lambda x: x.split('-')[0])

# integrity improvements printout
# print(f'YYYY-YYYY & YYYY-YY formatting: completeness of Date improved {len(dash_ranges)/(len(catalogue)):.0%}')
# print(f'remaining: {len(catalogue[catalogue.cleanedDate==0])/len(catalogue):.000%}')

# cleaning date ranges yyyyyyyyyy
# multirange = catalogue[catalogue.Date.str.contains('\d{9}[^\D]', regex=True)].index
# stashing [6:]
# catalogue.loc[multirange,'Date2'] = catalogue.loc[multirange,'Date'].apply(lambda x: str(x)[6:])
# keeping [0:4]
# catalogue.loc[multirange,'cleanedDate'] = catalogue.loc[multirange,'Date'].apply(lambda x: str(x)[0:4])
# discarding remainder [4:6]

# print(f'YYYYYYYYYY formatting: completeness of Date improved {len(multirange)/(len(catalogue)):.0%}')
# print(f'total improvements: {(len(catalogue[catalogue.cleanedDate!=0])-len(clean_dates))/(len(catalogue)):.0%}')
# sum(catalogue[catalogue['Date_test']==0]['Date'].value_counts())/len(catalogue)
# 98_541/len(catalogue)
# (len(catalogue)-(sum(catalogue[catalogue['Date_test']==0]['Date'].value_counts())+98541))
# daterange = catalogue[(catalogue.Date.str.len()==7)&(catalogue.Date.str.contains('-')==True)].index

# catalogue.loc[daterange,'cleanedDate'] = [i.split('-')[0] for i in catalogue.loc[daterange,'Date']]
# catalogue.loc[daterange,'Date2'] = [i.split('-')[1] for i in catalogue.loc[daterange,'Date']]
# print(f'completeness of Date improved {len(daterange)/(len(catalogue)):.0%}')
# print(f'total improvements: {(len(catalogue[catalogue.cleanedDate!=0])-len(clean_dates))/(len(catalogue)):.0%}')
# print(f'remaining: {len(catalogue[catalogue.cleanedDate==0])/len(catalogue):.0%}')

In [ ]:
str(set([i for i in n if i != '']))[1:-1].replace('\'','')

### Determining opportunities to impute Date <a class="anchor" id="date-imputation"></a>

## Extracting From Biographies <a class="anchor" id="bio-extraction"></a>
### [Extracting birthplace](#birthplace), if listed
### [Imputing nationalities](#impute-nationalities)
### [Creating `living` flag](#living)

### Extracting Birthplace <a class="anchor" id="birthplace"></a>

In [53]:
bio2ref = catalogue[catalogue.ArtistBio.str.contains(',')==True].index

catalogue.loc[:,'NationalityBio'] = catalogue.loc[:,'ArtistBio'].apply(lambda x: x.split(',')[0])
catalogue.loc[bio2ref,'Bio2'] = catalogue.loc[bio2ref,'ArtistBio'].apply(lambda x: x.split(',')[1])

In [54]:
catalogue[catalogue.Bio2.str.contains('born')==True].ArtistBio.value_counts()

American, born France. 1911–2010                3336
American, born Germany. 1886–1969               2657
American, born 1934                             1534
French, born Belarus. 1887–1985                 1161
American, born Lithuania. 1931–1978              821
                                                ... 
American, born England. 1906–1994                  1
French, born Romania 1922                          1
Brazilian, born Poland. 1921–2017                  1
British, born British Guiana now Guyana 1934       1
Malian, born 1953                                  1
Name: ArtistBio, Length: 2057, dtype: int64

In [55]:
bp_ref = catalogue[catalogue.ArtistBio.str.contains('born')==True].index

for i in bp_ref:
    output = list(set(re.findall('born\s\d{0,}\s{0,1}([A-Za-z]+)', catalogue.loc[i,'ArtistBio'])))
    
    if len(output) == 1:
        catalogue.loc[i,'Birthplace'] = output[0]
    elif len(output) == 0:
        catalogue.loc[i,'Birthplace'] = 'N/A'
    else:
        catalogue.loc[i,'Birthplace'] = [''.join(i) for i in output][0]
        
catalogue['Birthplace'].fillna('N/A', inplace=True)

KeyboardInterrupt: 

In [ ]:
[' '.join(i) for i in output][0]

In [ ]:
catalogue.Birthplace.value_counts()[20:40]

In [ ]:
# Answer
primes = [] # Set a list to catch prime values

for i in range(3, 2000): 
    # All statement evaluates to true if all of the iterables satisfy the criteria
    # If i divided by the existing primes(x) never has a remainder of 0
    if all(i % x != 0 for x in primes):
        # Append this number to the primes list
        primes.append(i)

sum(primes)

In [ ]:
catalogue[catalogue.Birthplace=="['Uruguay', 'American', 'Argentine']"]

In [ ]:
catalogue[(catalogue.Nationality==None)]

In [ ]:
catalogue['Nationality'].unique()

In [ ]:
catalogue['Bio2'].unique()[0:100]

In [ ]:
catalogue[catalogue.Birthplace.str.contains('\[')==True].Birthplace.unique()

### Impute Nationalities <a class="anchor" id="impute-nationalities"></a>

In [ ]:
# checking mismatches
len(catalogue[catalogue.Nationality!=catalogue.NationalityBio])

In [ ]:
# pulling top 10 donors, by volume
[print('{:,} items from {}'.format(catalogue.CreditLine.value_counts()[i], i)) for i in catalogue.CreditLine.value_counts().index[0:10]];

In [ ]:
# pulling remaining, most common formatting issues
catalogue[catalogue.cleanedDate==0].Date.value_counts()[0:10]

In [ ]:
catalogue[(catalogue.Date==0)&(catalogue.BeginDate.str.len()==4)]

In [ ]:
catalogue[(catalogue.Date==0)|(catalogue.Date==None)]

In [ ]:
# for i in catalogue.loc[ref, 'Date'].index:
#     f = catalogue.loc[i,'Date'][:4]
#     s = catalogue.loc[i,'Date'][4:]
    
#     catalogue.loc[i,'Date'] = f
#     catalogue.loc[i,'Date2'] = s

### Impute Deceased Year & `living` flag <a class="anchor" id="living"></a>

## Title EDA & Imputation <a class='anchor' id='title-eda'></a>

# Graveyard
## Turn back now!

[Evaluating `nulls`](#evaluating-nulls) is no longer relevant, as the dataset was limited to single-artist artworks. Storing in graveyard for potential future use.

In [ ]:
null_summary = pd.DataFrame(round(catalogue.isnull().sum()/len(catalogue),2), columns=['pct'])
null_summary = null_summary.reset_index(drop=False).rename(columns={'index':'Column'})
# [print(f"| {null_summary.loc[i,'Column']} | {null_summary.loc[i,'pct']} |") for i in null_summary[null_summary.pct==0].index]
# [print(f"| {null_summary.loc[i,'Column']} | {null_summary.loc[i,'pct']} |") for i in null_summary[null_summary.pct>0.1].index];
# [print(f"| {null_summary.loc[i,'Column']} | {null_summary.loc[i,'pct']} |") for i in null_summary[(null_summary.pct<=0.1)&(null_summary.pct!=0)].index];

### Evaluating `nulls` <a class='anchor' id='nulls'></a>
#### Complete Columns
- Artist (intervention)
- Title (further explored in [Title EDA](#title-eda))
- AccessionNumber (internal)
- Classification (internal)
- Department (internal)
- Object ID (internal)
- Cataloged (will be converted to bool)

#### Looking at those columns with high frequency of nulls (>10%)...
#### _Items to remove_
| Column | % Null | Notes |
| --- | --- | --- |
| URL | 0.33 | Not needed |
| ThumbnailURL | 0.40 | Not needed |
| Weight (kg) | 1.0 | Not needed |
| Seat Height (cm) | 1.0 | Not needed |
#### _Remaining_
| Column | % Null | Notes |
| --- | --- | --- |
| Circumference (cm) | 1.0 | Likely related to `Medium` |
| Depth (cm) | 0.89 | Likely related to `Medium` |
| Diameter (cm) | 0.99 | Likely related to `Medium` |
| Height (cm) | 0.12 | Possible imputation w/`Dimensions` |
| Length (cm) | 0.99 | Possible imputation w/`Dimensions` |
| Width (cm) | 0.13 | Possible imputation w/`Dimensions` |
| Duration (sec.) | 0.99 | Dependent on `Medium` |
#### Looking at those columns with low frequency nulls (<10%)...
#### _Linked to `null` Artist value_
| Column | % Null | Notes |
| --- | --- | --- |
| Artist | 0.01 | Removed. Represented .008 / 0.8% of all records. |
| ConstituentID | 0.01 | Dropped w/null artist |
| Nationality | 0.01 | Dropped w/null artist |
| BeginDate | 0.01 | Dropped w/null artist |
| EndDate | 0.01 | Dropped w/null artist |
| Gender | 0.01 | Dropped w/null artist |
#### _Not linked to `null` Artist value_
| Column | % Null | Notes |
| --- | --- | --- |
| ArtistBio | 0.03 | Will investigate further at later stage |
| CreditLine | 0.01 | `null` may have significance/meaning (acquisition) |
| Date | 0.01 | Will investigate overlap w/nulls in `BeginDate` and `EndDate` |
| DateAcquired | 0.05 | Internal, will follow up as possibly useful for further analysis |
| Dimensions | 0.06 | Possibly linked to `Medium` type |
| Medium | 0.07 | Will investigate further at later stage |

#### Deduping within rows <a class="anchor" id="deduping-within"></a>
_Many of our features are formatted in different ways, with duplicate values within, like so:_
- Gender: `(Male) (Male) (Male) (Male) (Male) (Male) (Male) (Male) (Male) (Female) (Male) (Male) ()`
- Nationality: `(Spanish) (Cuban) (Spanish) (Spanish)`
`strip_punct` handled step 1 - results:
- Gender: `Male Female`
- Nationality: `Spanish Cuban`
Last step is to dedupe within values.

In [ ]:
def dedupe(input: str) -> str:
    '''
    Takes in a string and processes it to return unique gender(s).
    '''
    n = [i.capitalize() for i in input.strip().split(' ')]
    if len(set(n)) == 1:
        if n[0] is None or n[0] == '':
            return 'N/A'
        return list(set(n))[0]
    else:
        return str(set([i for i in n if i != '']))[1:-1].replace('\'','')

standard = ['Male','Female']
len(catalogue[(catalogue['Gender']!='Female')& (catalogue['Gender']!='Male')&(catalogue['Artist'].str.contains(','))]), len(catalogue[(catalogue['Gender']!='Female')& (catalogue['Gender']!='Male')&(catalogue['Artist'].str.contains(',')==False)])

In [ ]:
catalogue[(catalogue['Gender']!='Female')& (catalogue['Gender']!='Male')&(catalogue['Artist'].str.contains(',')==False)]

In [ ]:
# gender requires deduping within row values
catalogue['Gender'] = catalogue['Gender'].apply(lambda x: dedupe(x))

Parsing nationalities

In [ ]:
output = ''

[output.join(i) for i in list(set([i.replace(')','').replace('(','') for i in re.split('\)\s\(',"(American) (American) (French)") if len(i) > 0 ]))]
print(output)


In [ ]:
def parse_multi_nat(input: str) -> str:
    return ' '.join(list(set([i.replace(')','').replace('(','').replace('\t','') for i in re.split('\)\s\(',input) if len(i) > 0 ]))).strip()

In [ ]:
# # testing
# parse_multi_nat('(American) (American) (Brazilian) (French) () (American)')
# parse_multi_nat('(American) (American) () (American)')
parse_multi_nat('(German) (Swedish) (German)	')

In [ ]:
multi_nationality = catalogue[catalogue.Nationality.str.contains('\) ')].index
catalogue.loc[multi_nationality,'cleanedNationality'] = catalogue.loc[multi_nationality,'Nationality'].apply(lambda x: parse_multi_nat(x))

In [ ]:
clean_nats = catalogue[catalogue.cleanedNationality.isnull()].index
catalogue.loc[clean_nats,'cleanedNationality'] = catalogue.loc[clean_nats,'Nationality']
# catalogue.loc[:,'cleanedNationality'] = catalogue.loc[:,'cleanedNationality'].str.replace(' Nationality unkown ','')

In [ ]:
catalogue.cleanedNationality.unique()

In [ ]:
# # improved method
# #processing dates formatted YYYYYYYY
# nodash = catalogue[(catalogue.Date.str.len()==8)&(catalogue.Date.str.contains('-')==False)].index
# catalogue.loc[nodash,'cleanedDate'] = [int(i[0:4]) for i in catalogue.loc[nodash,'Date']]
# catalogue.loc[nodash,'Date2'] = [int(i[4:]) for i in catalogue.loc[nodash,'Date']]

# print(f'completeness of Date improved {len(nodash)/(len(catalogue)):.0%}')
# print(f'total improvements: {(len(catalogue[catalogue.cleanedDate!=0])-len(clean_dates))/(len(catalogue)):.0%}')
